# STEP 1: Load Preprocessed Data

In [634]:
import pandas as pd
import numpy as np

artists_df = pd.read_csv("/Users/hoon/Desktop/4060J_DataScience_Project/artists.csv")

'''
# Load the preprocessed data 
1) parse & organize -> 2) feature extraction (pretrained ResNet50) -> 3) PCA to reduce dimentionality into 100
'''
image_metadata_reduced_df = pd.read_pickle('/Users/hoon/Desktop/image_metadata_reduced.pkl')


# Normalize artist names in both DataFrames (Albrecht Dürer)
import unicodedata

def normalize_name(name):
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8').strip()

image_metadata_reduced_df['artist_name'] = image_metadata_reduced_df['artist_name'].apply(normalize_name)
artists_df['name'] = artists_df['name'].apply(normalize_name)

In [636]:
display(artists_df.head())
display(image_metadata_reduced_df)

,id,name,years,genre,nationality,bio,wikipedia,paintings
0,0,Amedeo Modigliani,1884 - 1920,Expressionism,Italian,Amedeo Clemente Modigliani (Italian pronunciat...,http://en.wikipedia.org/wiki/Amedeo_Modigliani,193
1,1,Vasiliy Kandinskiy,1866 - 1944,"Expressionism,Abstractionism",Russian,Wassily Wassilyevich Kandinsky (Russian: Васи́...,http://en.wikipedia.org/wiki/Wassily_Kandinsky,88
2,2,Diego Rivera,1886 - 1957,"Social Realism,Muralism",Mexican,Diego María de la Concepción Juan Nepomuceno E...,http://en.wikipedia.org/wiki/Diego_Rivera,70
3,3,Claude Monet,1840 - 1926,Impressionism,French,Oscar-Claude Monet (; French: [klod mɔnɛ]; 14 ...,http://en.wikipedia.org/wiki/Claude_Monet,73
4,4,Rene Magritte,1898 - 1967,"Surrealism,Impressionism",Belgian,René François Ghislain Magritte (French: [ʁəne...,http://en.wikipedia.org/wiki/René_Magritte,194


,file_name,artist_name,features,artist_label
0,Gustav_Klimt_113.jpg,Gustav Klimt,"[-104.051254, -30.065239, 40.834503, -27.93637...",19
1,Vincent_van_Gogh_388.jpg,Vincent van Gogh,"[-42.690506, 50.457077, -31.054712, 49.034294,...",48
2,Amedeo_Modigliani_24.jpg,Amedeo Modigliani,"[85.80638, 74.72588, 21.94299, -97.160614, 6.6...",2
3,Edgar_Degas_455.jpg,Edgar Degas,"[110.57151, -27.310974, -106.109825, -26.60741...",10
4,Edgar_Degas_333.jpg,Edgar Degas,"[128.26991, -22.02461, -65.37852, -10.970957, ...",10
...,...,...,...,...
8350,Mikhail_Vrubel_116.jpg,Mikhail Vrubel,"[-112.82036, -11.718504, -2.0028427, -15.04778...",32
8351,Joan_Miro_51.jpg,Joan Miro,"[-69.65716, 75.916565, -17.81109, -51.636993, ...",27
8352,Frida_Kahlo_10.jpg,Frida Kahlo,"[53.062565, 15.8918295, -43.91319, -41.854576,...",16
8353,Vincent_van_Gogh_391.jpg,Vincent van Gogh,"[-88.60199, -56.594482, -37.02463, 22.391777, ...",48


In [638]:
# Merge the genre/style information into the image metadata DataFrame
image_metadata_reduced_df = image_metadata_reduced_df.merge(
    artists_df[['name', 'genre']],  # Use the genre or style column from artists.csv
    left_on='artist_name',
    right_on='name',
    how='left'
)

# Drop the duplicate 'name' column
image_metadata_reduced_df.drop(columns=['name'], inplace=True)

# Verify the merged DataFrame
display(image_metadata_reduced_df)

,file_name,artist_name,features,artist_label,genre
0,Gustav_Klimt_113.jpg,Gustav Klimt,"[-104.051254, -30.065239, 40.834503, -27.93637...",19,"Symbolism,Art Nouveau"
1,Vincent_van_Gogh_388.jpg,Vincent van Gogh,"[-42.690506, 50.457077, -31.054712, 49.034294,...",48,Post-Impressionism
2,Amedeo_Modigliani_24.jpg,Amedeo Modigliani,"[85.80638, 74.72588, 21.94299, -97.160614, 6.6...",2,Expressionism
3,Edgar_Degas_455.jpg,Edgar Degas,"[110.57151, -27.310974, -106.109825, -26.60741...",10,Impressionism
4,Edgar_Degas_333.jpg,Edgar Degas,"[128.26991, -22.02461, -65.37852, -10.970957, ...",10,Impressionism
...,...,...,...,...,...
8350,Mikhail_Vrubel_116.jpg,Mikhail Vrubel,"[-112.82036, -11.718504, -2.0028427, -15.04778...",32,Symbolism
8351,Joan_Miro_51.jpg,Joan Miro,"[-69.65716, 75.916565, -17.81109, -51.636993, ...",27,Surrealism
8352,Frida_Kahlo_10.jpg,Frida Kahlo,"[53.062565, 15.8918295, -43.91319, -41.854576,...",16,"Primitivism,Surrealism"
8353,Vincent_van_Gogh_391.jpg,Vincent van Gogh,"[-88.60199, -56.594482, -37.02463, 22.391777, ...",48,Post-Impressionism


In [640]:
# Ensure the 'genre' column exists and has no NaN values
if 'genre' in image_metadata_reduced_df.columns:
    image_metadata_reduced_df['genre'] = image_metadata_reduced_df['genre'].fillna("")
else:
    raise KeyError("The 'genre' column is missing from the DataFrame.")

# Create the 'genre_list' column
image_metadata_reduced_df['genre_list'] = image_metadata_reduced_df['genre'].apply(lambda x: x.split(","))

In [642]:
# Check the first few rows of the DataFrame
#display(image_metadata_reduced_df[['genre', 'genre_list']])

In [644]:
from sklearn.preprocessing import MultiLabelBinarizer

# One-hot encode the genres
mlb = MultiLabelBinarizer()
one_hot_genres = mlb.fit_transform(image_metadata_reduced_df['genre_list'])

# Add the one-hot encoded genres as columns
genre_columns = mlb.classes_
for idx, genre in enumerate(genre_columns):
    image_metadata_reduced_df[genre] = one_hot_genres[:, idx]

# Example output
display(image_metadata_reduced_df)

,file_name,artist_name,features,artist_label,genre,genre_list,Abstract Expressionism,Abstractionism,Art Nouveau,Baroque,...,Pop Art,Post-Impressionism,Primitivism,Proto Renaissance,Realism,Romanticism,Social Realism,Suprematism,Surrealism,Symbolism
0,Gustav_Klimt_113.jpg,Gustav Klimt,"[-104.051254, -30.065239, 40.834503, -27.93637...",19,"Symbolism,Art Nouveau","[Symbolism, Art Nouveau]",0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,Vincent_van_Gogh_388.jpg,Vincent van Gogh,"[-42.690506, 50.457077, -31.054712, 49.034294,...",48,Post-Impressionism,[Post-Impressionism],0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
2,Amedeo_Modigliani_24.jpg,Amedeo Modigliani,"[85.80638, 74.72588, 21.94299, -97.160614, 6.6...",2,Expressionism,[Expressionism],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Edgar_Degas_455.jpg,Edgar Degas,"[110.57151, -27.310974, -106.109825, -26.60741...",10,Impressionism,[Impressionism],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Edgar_Degas_333.jpg,Edgar Degas,"[128.26991, -22.02461, -65.37852, -10.970957, ...",10,Impressionism,[Impressionism],0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8350,Mikhail_Vrubel_116.jpg,Mikhail Vrubel,"[-112.82036, -11.718504, -2.0028427, -15.04778...",32,Symbolism,[Symbolism],0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
8351,Joan_Miro_51.jpg,Joan Miro,"[-69.65716, 75.916565, -17.81109, -51.636993, ...",27,Surrealism,[Surrealism],0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
8352,Frida_Kahlo_10.jpg,Frida Kahlo,"[53.062565, 15.8918295, -43.91319, -41.854576,...",16,"Primitivism,Surrealism","[Primitivism, Surrealism]",0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
8353,Vincent_van_Gogh_391.jpg,Vincent van Gogh,"[-88.60199, -56.594482, -37.02463, 22.391777, ...",48,Post-Impressionism,[Post-Impressionism],0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [646]:
from sklearn.model_selection import train_test_split

# Extract features and genres (as one-hot encoded labels)
X = np.stack(image_metadata_reduced_df['features'].values)  # Feature matrix
y_genre = image_metadata_reduced_df[genre_columns].values  # One-hot encoded genres

# Add indices to track the split
indices = np.arange(len(X))  # Create indices for the full dataset

# Split the data into training and testing sets, including the indices
X_train, X_test, y_train, y_test, train_indices, test_indices = train_test_split(
    X, y_genre, indices, test_size=0.2, random_state=42
)

# STEP 2: Implementation

## MODEL 1: Simple Neural Networks (Multi Layer Perceptron) <1>
Two hidden layers

In [650]:
class SimpleNeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * 0.01,
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * 0.01,
            "W3": np.random.randn(hidden_sizes[1], output_size) * 0.01,
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples
 
    def forward(self, X):
        # Forward propagation
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)  # Sigmoid activation for multi-label classification

        return self.A3

    def backward(self, X, y_true, y_pred):
        # Backward propagation
        n_samples = y_true.shape[0]

        # Gradients for the output layer
        dZ3 = y_pred - y_true  # Binary cross-entropy gradient
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        # Gradients for the second hidden layer
        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        # Gradients for the first hidden layer
        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        # Update weights and biases
        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1

        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2

        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            # Shuffle the data
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            # Mini-batch gradient descent
            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]

                # Forward and backward propagation
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            # Compute loss
            y_pred_full = self.forward(X)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X, threshold=0.5):
        y_pred = self.forward(X)
        max_probs = np.max(y_pred, axis=1) # Highest probability for each sample
        max_labels = np.argmax(y_pred, axis=1) # Index of the highest probability
        return max_probs, max_labels

In [652]:
##Train and Evaluate the Neural Network for Genre Classification
input_size = X_train.shape[1]
hidden_sizes = [512, 256]  # Two hidden layers with 512 and 256 neurons
output_size = y_train.shape[1]  # Number of genres

# Initialize and train the neural network
nn1 = SimpleNeuralNetwork(input_size, hidden_sizes, output_size, learning_rate=0.01)
nn1.train(X_train, y_train, epochs=30, batch_size=32)

# Evaluate on the test set
y_pred = nn1.predict(X_test)

Epoch 1/30, Loss: 3.0089
Epoch 2/30, Loss: 2.3156
Epoch 3/30, Loss: 1.8903
Epoch 4/30, Loss: 1.6310
Epoch 5/30, Loss: 1.5167
Epoch 6/30, Loss: 1.2284
Epoch 7/30, Loss: 1.0752
Epoch 8/30, Loss: 0.8934
Epoch 9/30, Loss: 0.7664
Epoch 10/30, Loss: 0.5620
Epoch 11/30, Loss: 0.4969
Epoch 12/30, Loss: 0.4553
Epoch 13/30, Loss: 0.3783
Epoch 14/30, Loss: 0.3229
Epoch 15/30, Loss: 0.2846
Epoch 16/30, Loss: 0.1472
Epoch 17/30, Loss: 0.0883
Epoch 18/30, Loss: 0.0628
Epoch 19/30, Loss: 0.0502
Epoch 20/30, Loss: 0.0362
Epoch 21/30, Loss: 0.0260
Epoch 22/30, Loss: 0.0222
Epoch 23/30, Loss: 0.0192
Epoch 24/30, Loss: 0.0171
Epoch 25/30, Loss: 0.0153
Epoch 26/30, Loss: 0.0142
Epoch 27/30, Loss: 0.0129
Epoch 28/30, Loss: 0.0119
Epoch 29/30, Loss: 0.0111
Epoch 30/30, Loss: 0.0103


In [654]:
# Evaluate on the test set
max_probs, predicted_genres_indices = nn1.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize counters
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions, actual genres, and test indices
for i, (prob, genre, actual_genre_list, test_idx) in enumerate(zip(max_probs, predicted_genres, y_test, test_indices)):
    # Map one-hot encoded actual genres to their names
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre_list) if val == 1]
    
    # Extract the painter's name for the test sample
    painter = image_metadata_reduced_df.iloc[test_idx]['artist_name']
    
    # Check if the predicted genre is in the actual genres
    if genre in actual_genres:
        correct_predictions += 1

    # Display results for each test sample
    print(f"Test Sample {i + 1}:")
    print(f"Test Data Painter: {painter}")
    print(f"Predicted Highest Probability: {prob:.9f}")
    print(f"Predicted Genre: {genre}")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct Prediction: {'Yes' if genre in actual_genres else 'No'}")
    print("-" * 30)

Test Sample 1:
Test Data Painter: Albrecht Durer
Predicted Highest Probability: 0.999999979
Predicted Genre: Northern Renaissance
Actual Genres: ['Northern Renaissance']
Correct Prediction: Yes
------------------------------
Test Sample 2:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.358817259
Predicted Genre: Cubism
Actual Genres: ['Romanticism']
Correct Prediction: No
------------------------------
Test Sample 3:
Test Data Painter: Pierre-Auguste Renoir
Predicted Highest Probability: 0.836422648
Predicted Genre: Cubism
Actual Genres: ['Impressionism']
Correct Prediction: No
------------------------------
Test Sample 4:
Test Data Painter: Gustave Courbet
Predicted Highest Probability: 0.998670070
Predicted Genre: High Renaissance
Actual Genres: ['Realism']
Correct Prediction: No
------------------------------
Test Sample 5:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.999999899
Predicted Genre: Romanticism
Actual Genres: ['Romanticism']
Corr

In [656]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Simple Neural Network Accuracy: {accuracy:.2f}%")

Simple Neural Network Accuracy: 73.49%


# Model 2: Boosted Neural Network (Ensemble Learning) <2>
Boosting trains multiple neural networks sequentially, focusing on correcting the errors of the previous network.

In [659]:
class BoostedNeuralNetwork:
    def __init__(self, base_model_class, n_models, input_size, hidden_sizes, output_size, learning_rate, gradient_clip_value=0.5):
        self.models = []
        self.gradient_clip_value = gradient_clip_value

        for _ in range(n_models):
            model = base_model_class(input_size, hidden_sizes, output_size, learning_rate)
            self.models.append(model)

    def train(self, X, y, epochs=10, batch_size=32, residual_threshold=1e-3):
        residuals = y
        for model_idx, model in enumerate(self.models):
            print(f"Training model {model_idx + 1}/{len(self.models)}")
            
            # Train the current model
            model.train(X, residuals, epochs, batch_size, self.gradient_clip_value)
            
            # Compute predictions
            predictions = model.forward(X)
            
            # Update residuals
            residuals = residuals - predictions
            
            # Scale residuals to prevent numerical instability
            residual_norm = np.linalg.norm(residuals, axis=0, keepdims=True)
            if np.all(residual_norm < residual_threshold):
                print("Residuals below threshold. Stopping further training.")
                break
            residuals = residuals / residual_norm

    def predict(self, X):
        # Aggregate predictions from all models
        ensemble_predictions = np.zeros_like(self.models[0].forward(X))
        for model in self.models:
            ensemble_predictions += model.forward(X)
        return ensemble_predictions / len(self.models)  # Average predictions


class SimpleNeuralNetwork2:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate):
        self.learning_rate = learning_rate
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * 0.01,
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * 0.01,
            "W3": np.random.randn(hidden_sizes[1], output_size) * 0.01,
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def forward(self, X):
        # Forward propagation
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)
        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)
        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)
        return self.A3

    def backward(self, X, y_true, y_pred, gradient_clip_value):
        # Backward propagation
        n_samples = y_true.shape[0]

        dZ3 = y_pred - y_true
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        # Gradient clipping
        dW3 = np.clip(dW3, -gradient_clip_value, gradient_clip_value)
        dW2 = np.clip(dW2, -gradient_clip_value, gradient_clip_value)
        dW1 = np.clip(dW1, -gradient_clip_value, gradient_clip_value)

        # Update weights and biases
        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1
        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2
        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3

    def train(self, X, y, epochs, batch_size, gradient_clip_value):
        n_samples = X.shape[0]
        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]
            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred, gradient_clip_value)
            y_pred_full = self.forward(X)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

In [661]:
# Create and train boosted neural networks
boosted_nn = BoostedNeuralNetwork(SimpleNeuralNetwork2, n_models=3, input_size=X_train.shape[1], hidden_sizes=[512, 256], output_size=y_train.shape[1], learning_rate=0.001)
boosted_nn.train(X_train, y_train, epochs=30, batch_size=32)

Training model 1/3
Epoch 1/30, Loss: 4.1971
Epoch 2/30, Loss: 4.0265
Epoch 3/30, Loss: 3.8274
Epoch 4/30, Loss: 3.6043
Epoch 5/30, Loss: 3.3970
Epoch 6/30, Loss: 3.2224
Epoch 7/30, Loss: 3.0786
Epoch 8/30, Loss: 2.9569
Epoch 9/30, Loss: 2.8461
Epoch 10/30, Loss: 2.7426
Epoch 11/30, Loss: 2.6504
Epoch 12/30, Loss: 2.5603
Epoch 13/30, Loss: 2.4851
Epoch 14/30, Loss: 2.4043
Epoch 15/30, Loss: 2.3335
Epoch 16/30, Loss: 2.2673
Epoch 17/30, Loss: 2.2166
Epoch 18/30, Loss: 2.1503
Epoch 19/30, Loss: 2.0924
Epoch 20/30, Loss: 2.0401
Epoch 21/30, Loss: 1.9899
Epoch 22/30, Loss: 1.9326
Epoch 23/30, Loss: 1.8886
Epoch 24/30, Loss: 1.8414
Epoch 25/30, Loss: 1.7963
Epoch 26/30, Loss: 1.7570
Epoch 27/30, Loss: 1.7066
Epoch 28/30, Loss: 1.6624
Epoch 29/30, Loss: 1.6259
Epoch 30/30, Loss: 1.5903
Training model 2/3
Epoch 1/30, Loss: 0.1027
Epoch 2/30, Loss: 0.0357
Epoch 3/30, Loss: 0.0192
Epoch 4/30, Loss: 0.0117
Epoch 5/30, Loss: 0.0070
Epoch 6/30, Loss: 0.0037
Epoch 7/30, Loss: 0.0010
Epoch 8/30, Loss

In [663]:
# Predict probabilities for the test set
boosted_predictions = boosted_nn.predict(X_test)  # Shape: (n_samples, num_classes)

# Get the highest probability and corresponding genres
max_probs = np.max(boosted_predictions, axis=1)  # Highest probability for each sample
predicted_genres_indices = np.argmax(boosted_predictions, axis=1)  # Index of the highest probability

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize counters
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions, actual genres, and test indices
for i, (prob, genre, actual_genre_list, test_idx) in enumerate(zip(max_probs, predicted_genres, y_test, test_indices)):
    # Map one-hot encoded actual genres to their names
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre_list) if val == 1]
    
    # Extract the painter's name for the test sample
    painter = image_metadata_reduced_df.iloc[test_idx]['artist_name']
    
    # Check if the predicted genre is in the actual genres
    if genre in actual_genres:
        correct_predictions += 1

    # Display results for each test sample
    print(f"Test Sample {i + 1}:")
    print(f"Test Data Painter: {painter}")
    print(f"Predicted Highest Probability: {prob:.9f}")
    print(f"Predicted Genre: {genre}")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct Prediction: {'Yes' if genre in actual_genres else 'No'}")
    print("-" * 30)

Test Sample 1:
Test Data Painter: Albrecht Durer
Predicted Highest Probability: 0.332142952
Predicted Genre: Northern Renaissance
Actual Genres: ['Northern Renaissance']
Correct Prediction: Yes
------------------------------
Test Sample 2:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.080090696
Predicted Genre: Primitivism
Actual Genres: ['Romanticism']
Correct Prediction: No
------------------------------
Test Sample 3:
Test Data Painter: Pierre-Auguste Renoir
Predicted Highest Probability: 0.128501161
Predicted Genre: Post-Impressionism
Actual Genres: ['Impressionism']
Correct Prediction: No
------------------------------
Test Sample 4:
Test Data Painter: Gustave Courbet
Predicted Highest Probability: 0.257619864
Predicted Genre: High Renaissance
Actual Genres: ['Realism']
Correct Prediction: No
------------------------------
Test Sample 5:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.300101251
Predicted Genre: Romanticism
Actual Genres: ['R

In [665]:
# Calculate and display overall accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Boosted Neural Network Accuracy: {accuracy:.2f}%")

Boosted Neural Network Accuracy: 67.21%


## Model 3: Neural Network with Droupout <애매>

In [667]:
class SimpleNeuralNetworkWithDropout:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01, dropout_rate=0.5):
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate

        # Initialize weights and biases
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * 0.01,
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * 0.01,
            "W3": np.random.randn(hidden_sizes[1], output_size) * 0.01,
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def apply_dropout(self, A):
        mask = (np.random.rand(*A.shape) > self.dropout_rate) / (1 - self.dropout_rate)
        return A * mask

    def forward(self, X, training=True):
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)
        if training:
            self.A1 = self.apply_dropout(self.A1)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)
        if training:
            self.A2 = self.apply_dropout(self.A2)

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)

        return self.A3

    def backward(self, X, y_true, y_pred):
        n_samples = y_true.shape[0]

        dZ3 = y_pred - y_true
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1
        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2
        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]
        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            y_pred_full = self.forward(X, training=False)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X, training=False)
        max_probs = np.max(y_pred, axis=1)
        max_labels = np.argmax(y_pred, axis=1)
        return max_probs, max_labels
    

In [675]:
# Define model parameters
input_size = X_train.shape[1]  # Number of input features
hidden_sizes = [512, 256]      # Two hidden layers
output_size = y_train.shape[1]  # Number of output genres
learning_rate = 0.01           # Learning rate
dropout_rate = 0.2             # Dropout rate

# Initialize the neural network with dropout
nn_with_dropout = SimpleNeuralNetworkWithDropout(input_size, hidden_sizes, output_size, learning_rate, dropout_rate)

# Train the neural network
nn_with_dropout.train(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30, Loss: 2.9942
Epoch 2/30, Loss: 2.2853
Epoch 3/30, Loss: 1.9492
Epoch 4/30, Loss: 1.7128
Epoch 5/30, Loss: 1.5279
Epoch 6/30, Loss: 1.3912
Epoch 7/30, Loss: 1.2378
Epoch 8/30, Loss: 1.0905
Epoch 9/30, Loss: 0.9503
Epoch 10/30, Loss: 0.9014
Epoch 11/30, Loss: 0.8677
Epoch 12/30, Loss: 0.7208
Epoch 13/30, Loss: 0.6565
Epoch 14/30, Loss: 0.5152
Epoch 15/30, Loss: 0.4917
Epoch 16/30, Loss: 0.4752
Epoch 17/30, Loss: 0.4341
Epoch 18/30, Loss: 0.4405
Epoch 19/30, Loss: 0.4227
Epoch 20/30, Loss: 0.4644
Epoch 21/30, Loss: 0.2866
Epoch 22/30, Loss: 0.2486
Epoch 23/30, Loss: 0.2666
Epoch 24/30, Loss: 0.2764
Epoch 25/30, Loss: 0.2385
Epoch 26/30, Loss: 0.2170
Epoch 27/30, Loss: 0.1407
Epoch 28/30, Loss: 0.1701
Epoch 29/30, Loss: 0.3168
Epoch 30/30, Loss: 0.1300


In [677]:
# Get predictions on the test set
max_probs, predicted_genres_indices = nn_with_dropout.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize counters for evaluation
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions and compare with actual genres
for i, (prob, genre_idx, actual_genre) in enumerate(zip(max_probs, predicted_genres_indices, y_test)):
    predicted_genre = genre_columns[genre_idx]
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre) if val == 1]
    
    # Check if the predicted genre is in the actual genres
    is_correct = predicted_genre in actual_genres
    if is_correct:
        correct_predictions += 1

    # Display the results
    print(f"Sample {i + 1}:")
    print(f"Predicted Genre: {predicted_genre} (Prob: {prob:.9f})")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct: {'Yes' if is_correct else 'No'}")
    print("-" * 30)

Sample 1:
Predicted Genre: Northern Renaissance (Prob: 1.000000000)
Actual Genres: ['Northern Renaissance']
Correct: Yes
------------------------------
Sample 2:
Predicted Genre: Post-Impressionism (Prob: 0.481315985)
Actual Genres: ['Romanticism']
Correct: No
------------------------------
Sample 3:
Predicted Genre: Cubism (Prob: 0.210237737)
Actual Genres: ['Impressionism']
Correct: No
------------------------------
Sample 4:
Predicted Genre: High Renaissance (Prob: 0.999841703)
Actual Genres: ['Realism']
Correct: No
------------------------------
Sample 5:
Predicted Genre: Romanticism (Prob: 0.999992562)
Actual Genres: ['Romanticism']
Correct: Yes
------------------------------
Sample 6:
Predicted Genre: Post-Impressionism (Prob: 0.993797937)
Actual Genres: ['Post-Impressionism', 'Symbolism']
Correct: Yes
------------------------------
Sample 7:
Predicted Genre: Baroque (Prob: 0.999999981)
Actual Genres: ['Baroque']
Correct: Yes
------------------------------
Sample 8:
Predicted Gen

In [679]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Dropout Dural Network Accuracy: {accuracy:.2f}%")

Dropout Dural Network Accuracy: 73.67%


# MODEL 4: With He Inititialization
This can help the network converge faster and reduce sensitivity to initial weights, making training more stable.

In [710]:
import numpy as np

class NeuralNetworkWithHeInit:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01):
        self.learning_rate = learning_rate

        # Initialize weights and biases using He Initialization
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * np.sqrt(2 / input_size),
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * np.sqrt(2 / hidden_sizes[0]),
            "W3": np.random.randn(hidden_sizes[1], output_size) * np.sqrt(2 / hidden_sizes[1]),
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    '''
    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))
    '''

    def sigmoid(self, Z):
        Z = np.clip(Z, -500, 500)  # Prevent overflow
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def forward(self, X):
        # Forward propagation
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)  # Sigmoid activation for multi-label classification

        return self.A3

    def backward(self, X, y_true, y_pred):
        # Backward propagation
        n_samples = y_true.shape[0]

        dZ3 = y_pred - y_true  # Binary cross-entropy gradient
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        # Update weights and biases
        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1
        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2
        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            y_pred_full = self.forward(X)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        max_probs = np.max(y_pred, axis=1)  # Highest probability for each sample
        max_labels = np.argmax(y_pred, axis=1)  # Index of the highest probability
        return max_probs, max_labels

In [ ]:
# Define model parameters
input_size = X_train.shape[1]  # Number of input features
hidden_sizes = [512, 256]      # Two hidden layers
output_size = y_train.shape[1]  # Number of output genres
learning_rate = 0.01           # Learning rate

# Initialize and train the neural network
nn_he_init = NeuralNetworkWithHeInit(input_size, hidden_sizes, output_size, learning_rate)
nn_he_init.train(X_train, y_train, epochs=30, batch_size=32)

In [715]:
# Predict on test data
max_probs, predicted_genres_indices = nn_he_init.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize counters for evaluation
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions and compare with actual genres
for i, (prob, genre_idx, actual_genre) in enumerate(zip(max_probs, predicted_genres_indices, y_test)):
    predicted_genre = genre_columns[genre_idx]
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre) if val == 1]
    
    # Check if the predicted genre is in the actual genres
    is_correct = predicted_genre in actual_genres
    if is_correct:
        correct_predictions += 1

    # Display the results
    print(f"Sample {i + 1}:")
    print(f"Predicted Genre: {predicted_genre} (Prob: {prob:.9f})")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct: {'Yes' if is_correct else 'No'}")
    print("-" * 30)

Sample 1:
Predicted Genre: Northern Renaissance (Prob: 0.988762016)
Actual Genres: ['Northern Renaissance']
Correct: Yes
------------------------------
Sample 2:
Predicted Genre: Northern Renaissance (Prob: 0.693259106)
Actual Genres: ['Romanticism']
Correct: No
------------------------------
Sample 3:
Predicted Genre: Symbolism (Prob: 0.476289353)
Actual Genres: ['Impressionism']
Correct: No
------------------------------
Sample 4:
Predicted Genre: Baroque (Prob: 0.512034981)
Actual Genres: ['Realism']
Correct: No
------------------------------
Sample 5:
Predicted Genre: Romanticism (Prob: 1.000000000)
Actual Genres: ['Romanticism']
Correct: Yes
------------------------------
Sample 6:
Predicted Genre: Post-Impressionism (Prob: 0.877527065)
Actual Genres: ['Post-Impressionism', 'Symbolism']
Correct: Yes
------------------------------
Sample 7:
Predicted Genre: Baroque (Prob: 0.904216532)
Actual Genres: ['Baroque']
Correct: Yes
------------------------------
Sample 8:
Predicted Genre: 

In [717]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"He Initialization Accuracy: {accuracy:.2f}%")

Accuracy: 59.07%


## MODEL 5: Neural Network with Adam Optimizer
Here’s the key idea:  
- Adam combines the benefits of Momentum Optimization and RMSProp.
- It uses exponential moving averages of the gradients and the squared gradients to adaptively adjust the learning rate for each weight.
- This approach can handle noisy gradients and sparse data, which might be beneficial in your case.

In [721]:
class NeuralNetworkWithAdam:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-8):
        self.learning_rate = learning_rate
        self.beta1 = beta1  # Exponential decay rate for the first moment estimates
        self.beta2 = beta2  # Exponential decay rate for the second moment estimates
        self.epsilon = epsilon  # Small value to prevent division by zero

        # Initialize weights and biases
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * np.sqrt(2 / input_size),
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * np.sqrt(2 / hidden_sizes[0]),
            "W3": np.random.randn(hidden_sizes[1], output_size) * np.sqrt(2 / hidden_sizes[1]),
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

        # Initialize Adam parameters
        self.m_weights = {key: np.zeros_like(value) for key, value in self.weights.items()}
        self.v_weights = {key: np.zeros_like(value) for key, value in self.weights.items()}
        self.m_biases = {key: np.zeros_like(value) for key, value in self.biases.items()}
        self.v_biases = {key: np.zeros_like(value) for key, value in self.biases.items()}
        self.t = 0  # Time step for Adam

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        Z = np.clip(Z, -100, 100)  # Prevent overflow
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def forward(self, X):
        # Forward propagation
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)  # Sigmoid activation for multi-label classification

        return self.A3

    def adam_update(self, gradients, params, m_params, v_params):
        self.t += 1
        updated_params = {}
        for key in params.keys():
            # Update biased first moment estimate
            m_params[key] = self.beta1 * m_params[key] + (1 - self.beta1) * gradients[key]

            # Update biased second raw moment estimate
            v_params[key] = self.beta2 * v_params[key] + (1 - self.beta2) * (gradients[key] ** 2)

            # Correct bias in first moment
            m_hat = m_params[key] / (1 - self.beta1 ** self.t)

            # Correct bias in second raw moment
            v_hat = v_params[key] / (1 - self.beta2 ** self.t)

            # Update parameters
            updated_params[key] = params[key] - self.learning_rate * m_hat / (np.sqrt(v_hat) + self.epsilon)

        return updated_params, m_params, v_params

    def backward(self, X, y_true, y_pred):
        # Backward propagation
        n_samples = y_true.shape[0]

        dZ3 = y_pred - y_true  # Binary cross-entropy gradient
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        # Collect gradients
        gradients = {
            "W1": dW1,
            "W2": dW2,
            "W3": dW3,
            "b1": db1,
            "b2": db2,
            "b3": db3,
        }

        # Update weights and biases using Adam
        self.weights, self.m_weights, self.v_weights = self.adam_update(
            {key: gradients[key] for key in self.weights.keys()},
            self.weights,
            self.m_weights,
            self.v_weights,
        )
        self.biases, self.m_biases, self.v_biases = self.adam_update(
            {key: gradients[key] for key in self.biases.keys()},
            self.biases,
            self.m_biases,
            self.v_biases,
        )

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            y_pred_full = self.forward(X)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        max_probs = np.max(y_pred, axis=1)  # Highest probability for each sample
        max_labels = np.argmax(y_pred, axis=1)  # Index of the highest probability
        return max_probs, max_labels

In [723]:
# Define model parameters
input_size = X_train.shape[1]  # Number of input features
hidden_sizes = [512, 256]      # Two hidden layers
output_size = y_train.shape[1]  # Number of output genres
learning_rate = 0.001           # Lower learning rate for Adam

# Initialize and train the neural network
nn_adam = NeuralNetworkWithAdam(input_size, hidden_sizes, output_size, learning_rate)
nn_adam.train(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30, Loss: 5.1847
Epoch 2/30, Loss: 2.7548
Epoch 3/30, Loss: 1.9422
Epoch 4/30, Loss: 1.6177
Epoch 5/30, Loss: 1.2439
Epoch 6/30, Loss: 0.9991
Epoch 7/30, Loss: 0.7819
Epoch 8/30, Loss: 0.6510
Epoch 9/30, Loss: 0.5386
Epoch 10/30, Loss: 0.4344
Epoch 11/30, Loss: 0.3565
Epoch 12/30, Loss: 0.3739
Epoch 13/30, Loss: 0.2817
Epoch 14/30, Loss: 0.3668
Epoch 15/30, Loss: 0.3696
Epoch 16/30, Loss: 0.3822
Epoch 17/30, Loss: 0.3172
Epoch 18/30, Loss: 0.2566
Epoch 19/30, Loss: 0.2903
Epoch 20/30, Loss: 0.3022
Epoch 21/30, Loss: 0.2991
Epoch 22/30, Loss: 0.4026
Epoch 23/30, Loss: 0.3029
Epoch 24/30, Loss: 0.2549
Epoch 25/30, Loss: 0.2238
Epoch 26/30, Loss: 0.2464
Epoch 27/30, Loss: 0.2097
Epoch 28/30, Loss: 0.2284
Epoch 29/30, Loss: 0.2149
Epoch 30/30, Loss: 0.2717


In [727]:
# Predict on test data
max_probs, predicted_genres_indices = nn_he_init.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize co|unters for evaluation
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions and compare with actual genres
for i, (prob, genre_idx, actual_genre) in enumerate(zip(max_probs, predicted_genres_indices, y_test)):
    predicted_genre = genre_columns[genre_idx]
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre) if val == 1]
    
    # Check if the predicted genre is in the actual genres
    is_correct = predicted_genre in actual_genres
    if is_correct:
        correct_predictions += 1

    # Display the results
    print(f"Sample {i + 1}:")
    print(f"Predicted Genre: {predicted_genre} (Prob: {prob:.9f})")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct: {'Yes' if is_correct else 'No'}")
    print("-" * 30)

Sample 1:
Predicted Genre: Northern Renaissance (Prob: 0.988762016)
Actual Genres: ['Northern Renaissance']
Correct: Yes
------------------------------
Sample 2:
Predicted Genre: Northern Renaissance (Prob: 0.693259106)
Actual Genres: ['Romanticism']
Correct: No
------------------------------
Sample 3:
Predicted Genre: Symbolism (Prob: 0.476289353)
Actual Genres: ['Impressionism']
Correct: No
------------------------------
Sample 4:
Predicted Genre: Baroque (Prob: 0.512034981)
Actual Genres: ['Realism']
Correct: No
------------------------------
Sample 5:
Predicted Genre: Romanticism (Prob: 1.000000000)
Actual Genres: ['Romanticism']
Correct: Yes
------------------------------
Sample 6:
Predicted Genre: Post-Impressionism (Prob: 0.877527065)
Actual Genres: ['Post-Impressionism', 'Symbolism']
Correct: Yes
------------------------------
Sample 7:
Predicted Genre: Baroque (Prob: 0.904216532)
Actual Genres: ['Baroque']
Correct: Yes
------------------------------
Sample 8:
Predicted Genre: 

In [729]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 59.07%


## MODEL 6: Adding Residual Connections (버려)
adding residual connections can stabilize training and improve accuracy.

In [747]:
class ResidualNeuralNetwork:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.001):
        self.learning_rate = learning_rate

        # Initialize weights and biases
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * np.sqrt(2 / input_size),
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * np.sqrt(2 / hidden_sizes[0]),
            "W3": np.random.randn(hidden_sizes[1], output_size) * np.sqrt(2 / hidden_sizes[1]),
            "W_proj": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * np.sqrt(2 / hidden_sizes[0]),
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        Z = np.clip(Z, -100, 100)  # Prevent overflow
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def forward(self, X):
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A1_proj = np.dot(self.A1, self.weights["W_proj"])  # Project A1 to match Z2 size
        self.A2 = self.relu(self.Z2) + self.A1_proj  # Residual connection

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.sigmoid(self.Z3)

        return self.A3

    def backward(self, X, y_true, y_pred):
        n_samples = y_true.shape[0]

        dZ3 = y_pred - y_true
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1_proj = np.dot(dZ2, self.weights["W2"].T)
        dA1 = dA1_proj + np.dot(dZ2, self.weights["W_proj"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        dW_proj = np.dot(self.A1.T, dZ2) / n_samples

        # Gradient clipping
        dW1 = np.clip(dW1, -1, 1)
        dW2 = np.clip(dW2, -1, 1)
        dW3 = np.clip(dW3, -1, 1)
        dW_proj = np.clip(dW_proj, -1, 1)

        # Update weights and biases
        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1
        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2
        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3
        self.weights["W_proj"] -= self.learning_rate * dW_proj

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch)
                self.backward(X_batch, y_batch, y_pred)

            y_pred_full = self.forward(X)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X)
        max_probs = np.max(y_pred, axis=1)
        max_labels = np.argmax(y_pred, axis=1)
        return max_probs, max_labels

In [749]:
# Define model parameters
input_size = X_train.shape[1]  # Number of input features
hidden_sizes = [512, 256]      # Two hidden layers
output_size = y_train.shape[1]  # Number of output genres
learning_rate = 0.01           # Learning rate

# Initialize and train the neural network
nn_residual = ResidualNeuralNetwork(input_size, hidden_sizes, output_size, learning_rate)
nn_residual.train(X_train, y_train, epochs=30, batch_size=32)

Epoch 1/30, Loss: 66.9849
Epoch 2/30, Loss: 72.1348
Epoch 3/30, Loss: 27.4781
Epoch 4/30, Loss: 49.0266
Epoch 5/30, Loss: 59.3565
Epoch 6/30, Loss: 29.2234
Epoch 7/30, Loss: 33.8597
Epoch 8/30, Loss: 22.8306
Epoch 9/30, Loss: 51.8335
Epoch 10/30, Loss: 33.9794
Epoch 11/30, Loss: 36.5725
Epoch 12/30, Loss: 38.0676
Epoch 13/30, Loss: 36.8656
Epoch 14/30, Loss: 30.1909
Epoch 15/30, Loss: 17.5385
Epoch 16/30, Loss: 28.1878
Epoch 17/30, Loss: 14.8583
Epoch 18/30, Loss: 15.3351
Epoch 19/30, Loss: 15.4396
Epoch 20/30, Loss: 12.4814
Epoch 21/30, Loss: 15.1448
Epoch 22/30, Loss: 17.0134
Epoch 23/30, Loss: 16.7776
Epoch 24/30, Loss: 10.6222
Epoch 25/30, Loss: 10.9113
Epoch 26/30, Loss: 13.1106
Epoch 27/30, Loss: 11.2719
Epoch 28/30, Loss: 9.9275
Epoch 29/30, Loss: 10.1370
Epoch 30/30, Loss: 15.3887


In [755]:
# Predict on test data
max_probs, predicted_genres_indices = nn_residual.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize co|unters for evaluation
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions and compare with actual genres
for i, (prob, genre_idx, actual_genre) in enumerate(zip(max_probs, predicted_genres_indices, y_test)):
    predicted_genre = genre_columns[genre_idx]
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre) if val == 1]
    
    # Check if the predicted genre is in the actual genres
    is_correct = predicted_genre in actual_genres
    if is_correct:
        correct_predictions += 1

    # Display the results
    print(f"Sample {i + 1}:")
    print(f"Predicted Genre: {predicted_genre} (Prob: {prob:.9f})")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct: {'Yes' if is_correct else 'No'}")
    print("-" * 30)

Sample 1:
Predicted Genre: Northern Renaissance (Prob: 1.000000000)
Actual Genres: ['Northern Renaissance']
Correct: Yes
------------------------------
Sample 2:
Predicted Genre: Post-Impressionism (Prob: 1.000000000)
Actual Genres: ['Romanticism']
Correct: No
------------------------------
Sample 3:
Predicted Genre: Cubism (Prob: 1.000000000)
Actual Genres: ['Impressionism']
Correct: No
------------------------------
Sample 4:
Predicted Genre: High Renaissance (Prob: 1.000000000)
Actual Genres: ['Realism']
Correct: No
------------------------------
Sample 5:
Predicted Genre: Post-Impressionism (Prob: 1.000000000)
Actual Genres: ['Romanticism']
Correct: No
------------------------------
Sample 6:
Predicted Genre: Post-Impressionism (Prob: 1.000000000)
Actual Genres: ['Post-Impressionism', 'Symbolism']
Correct: Yes
------------------------------
Sample 7:
Predicted Genre: Baroque (Prob: 1.000000000)
Actual Genres: ['Baroque']
Correct: Yes
------------------------------
Sample 8:
Predict

In [757]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 55.72%


# MODEL 7: Deeper Neural Networks (버려)
Test with more features (no PCA applied).

In [759]:
# Load the unprocessed feature data
image_metadata_path = "/Users/hoon/Desktop/image_metadata.pkl"
image_metadata_df = pd.read_pickle(image_metadata_path)

# Ensure the 'artist_name' column is normalized
import unicodedata
def normalize_name(name):
    return unicodedata.normalize('NFKD', name).encode('ascii', 'ignore').decode('utf-8').strip()

image_metadata_df['artist_name'] = image_metadata_df['artist_name'].apply(normalize_name)

# Load artist metadata
artists_csv_path = "/Users/hoon/Desktop/4060J_DataScience_Project/artists.csv"
artists_df = pd.read_csv(artists_csv_path)
artists_df['name'] = artists_df['name'].apply(normalize_name)

# Merge genre/style information into the image metadata DataFrame
image_metadata_df = image_metadata_df.merge(
    artists_df[['name', 'genre']],  # Select 'name' and 'genre' columns from artists.csv
    left_on='artist_name',
    right_on='name',
    how='left'
)

# Drop the duplicate 'name' column
image_metadata_df.drop(columns=['name'], inplace=True)

# Ensure 'genre' column exists and has no NaN values
if 'genre' in image_metadata_df.columns:
    image_metadata_df['genre'] = image_metadata_df['genre'].fillna("")
else:
    raise KeyError("The 'genre' column is missing from the DataFrame.")

# Create 'genre_list' column from the 'genre' column
image_metadata_df['genre_list'] = image_metadata_df['genre'].apply(lambda x: x.split(","))

# One-hot encode the genres using MultiLabelBinarizer
mlb = MultiLabelBinarizer()
one_hot_genres = mlb.fit_transform(image_metadata_df['genre_list'])
genre_columns = mlb.classes_

# Add the one-hot encoded genres to the DataFrame
for idx, genre in enumerate(genre_columns):
    image_metadata_df[genre] = one_hot_genres[:, idx]

In [760]:
# Prepare the features and labels
X = np.stack(image_metadata_df['features'].values)  # Use raw features (not reduced by PCA)
y = image_metadata_df[genre_columns].values  # One-hot encoded genres

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verify shapes of the data
print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

X_train shape: (6684, 100352), y_train shape: (6684, 24)
X_test shape: (1671, 100352), y_test shape: (1671, 24)


In [763]:
### 4 hidden layers <테스트 해야함>
import numpy as np

class DeepNeuralNetworkWithDropout:
    def __init__(self, input_size, hidden_sizes, output_size, learning_rate=0.01, dropout_rate=0.5):
        self.learning_rate = learning_rate
        self.dropout_rate = dropout_rate

        # Initialize weights and biases
        self.weights = {
            "W1": np.random.randn(input_size, hidden_sizes[0]) * 0.01,
            "W2": np.random.randn(hidden_sizes[0], hidden_sizes[1]) * 0.01,
            "W3": np.random.randn(hidden_sizes[1], hidden_sizes[2]) * 0.01,
            "W4": np.random.randn(hidden_sizes[2], hidden_sizes[3]) * 0.01,
            "W5": np.random.randn(hidden_sizes[3], output_size) * 0.01,
        }
        self.biases = {
            "b1": np.zeros((1, hidden_sizes[0])),
            "b2": np.zeros((1, hidden_sizes[1])),
            "b3": np.zeros((1, hidden_sizes[2])),
            "b4": np.zeros((1, hidden_sizes[3])),
            "b5": np.zeros((1, output_size)),
        }

    def relu(self, Z):
        return np.maximum(0, Z)

    def relu_derivative(self, Z):
        return Z > 0

    def sigmoid(self, Z):
        return 1 / (1 + np.exp(-Z))

    def binary_cross_entropy_loss(self, y_pred, y_true):
        n_samples = y_true.shape[0]
        return -np.sum(y_true * np.log(y_pred + 1e-10) + (1 - y_true) * np.log(1 - y_pred + 1e-10)) / n_samples

    def dropout(self, A, training=True):
        if not training:
            return A  # No dropout during testing
        dropout_mask = np.random.rand(*A.shape) > self.dropout_rate
        return A * dropout_mask / (1 - self.dropout_rate)

    def forward(self, X, training=True):
        # Forward propagation with dropout
        self.Z1 = np.dot(X, self.weights["W1"]) + self.biases["b1"]
        self.A1 = self.relu(self.Z1)
        self.A1 = self.dropout(self.A1, training)

        self.Z2 = np.dot(self.A1, self.weights["W2"]) + self.biases["b2"]
        self.A2 = self.relu(self.Z2)
        self.A2 = self.dropout(self.A2, training)

        self.Z3 = np.dot(self.A2, self.weights["W3"]) + self.biases["b3"]
        self.A3 = self.relu(self.Z3)
        self.A3 = self.dropout(self.A3, training)

        self.Z4 = np.dot(self.A3, self.weights["W4"]) + self.biases["b4"]
        self.A4 = self.relu(self.Z4)
        self.A4 = self.dropout(self.A4, training)

        self.Z5 = np.dot(self.A4, self.weights["W5"]) + self.biases["b5"]
        self.A5 = self.sigmoid(self.Z5)  # Sigmoid activation for multi-label classification

        return self.A5

    def backward(self, X, y_true, y_pred):
        # Backward propagation
        n_samples = y_true.shape[0]

        dZ5 = y_pred - y_true  # Binary cross-entropy gradient
        dW5 = np.dot(self.A4.T, dZ5) / n_samples
        db5 = np.sum(dZ5, axis=0, keepdims=True) / n_samples

        dA4 = np.dot(dZ5, self.weights["W5"].T)
        dZ4 = dA4 * self.relu_derivative(self.Z4)
        dW4 = np.dot(self.A3.T, dZ4) / n_samples
        db4 = np.sum(dZ4, axis=0, keepdims=True) / n_samples

        dA3 = np.dot(dZ4, self.weights["W4"].T)
        dZ3 = dA3 * self.relu_derivative(self.Z3)
        dW3 = np.dot(self.A2.T, dZ3) / n_samples
        db3 = np.sum(dZ3, axis=0, keepdims=True) / n_samples

        dA2 = np.dot(dZ3, self.weights["W3"].T)
        dZ2 = dA2 * self.relu_derivative(self.Z2)
        dW2 = np.dot(self.A1.T, dZ2) / n_samples
        db2 = np.sum(dZ2, axis=0, keepdims=True) / n_samples

        dA1 = np.dot(dZ2, self.weights["W2"].T)
        dZ1 = dA1 * self.relu_derivative(self.Z1)
        dW1 = np.dot(X.T, dZ1) / n_samples
        db1 = np.sum(dZ1, axis=0, keepdims=True) / n_samples

        # Update weights and biases
        self.weights["W1"] -= self.learning_rate * dW1
        self.biases["b1"] -= self.learning_rate * db1
        self.weights["W2"] -= self.learning_rate * dW2
        self.biases["b2"] -= self.learning_rate * db2
        self.weights["W3"] -= self.learning_rate * dW3
        self.biases["b3"] -= self.learning_rate * db3
        self.weights["W4"] -= self.learning_rate * dW4
        self.biases["b4"] -= self.learning_rate * db4
        self.weights["W5"] -= self.learning_rate * dW5
        self.biases["b5"] -= self.learning_rate * db5

    def train(self, X, y, epochs=20, batch_size=32):
        n_samples = X.shape[0]

        for epoch in range(epochs):
            indices = np.arange(n_samples)
            np.random.shuffle(indices)
            X = X[indices]
            y = y[indices]

            for i in range(0, n_samples, batch_size):
                X_batch = X[i:i + batch_size]
                y_batch = y[i:i + batch_size]
                y_pred = self.forward(X_batch, training=True)
                self.backward(X_batch, y_batch, y_pred)

            y_pred_full = self.forward(X, training=False)
            loss = self.binary_cross_entropy_loss(y_pred_full, y)
            print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss:.4f}")

    def predict(self, X):
        y_pred = self.forward(X, training=False)
        max_probs = np.max(y_pred, axis=1)  # Highest probability for each sample
        max_labels = np.argmax(y_pred, axis=1)  # Index of the highest probability
        return max_probs, max_labels

In [765]:
# Define model parameters
input_size = X_train.shape[1]  # Number of input features
hidden_sizes = [1024, 512, 256, 128]  # Four hidden layers
output_size = y_train.shape[1]  # Number of output genres
learning_rate = 0.01  # Learning rate
dropout_rate = 0.2  # Dropout rate

# Initialize and train the neural network
nn_dropout = DeepNeuralNetworkWithDropout(input_size, hidden_sizes, output_size, learning_rate, dropout_rate)
nn_dropout.train(X_train, y_train, epochs=30, batch_size=32)

KeyboardInterrupt: 

In [581]:
# Predict on test data
max_probs, predicted_genres_indices = nn_adam.predict(X_test)

# Map predicted genre indices to genre names
predicted_genres = [genre_columns[idx] for idx in predicted_genres_indices]

# Initialize counters for evaluation
correct_predictions = 0
total_predictions = len(y_test)

# Iterate through predictions and compare with actual genres
for i, (prob, genre_idx, actual_genre) in enumerate(zip(max_probs, predicted_genres_indices, y_test)):
    predicted_genre = genre_columns[genre_idx]
    actual_genres = [genre_columns[idx] for idx, val in enumerate(actual_genre) if val == 1]
    
    # Check if the predicted genre is in the actual genres
    is_correct = predicted_genre in actual_genres
    if is_correct:
        correct_predictions += 1

    # Display the results
    print(f"Sample {i + 1}:")
    print(f"Predicted Genre: {predicted_genre} (Prob: {prob:.9f})")
    print(f"Actual Genres: {actual_genres}")
    print(f"Correct: {'Yes' if is_correct else 'No'}")
    print("-" * 30)

Test Sample 1:
Test Data Painter: Albrecht Durer
Predicted Highest Probability: 0.999943283
Predicted Genre: Northern Renaissance
Actual Genres: ['Northern Renaissance']
Correct Prediction: Yes
------------------------------
Test Sample 2:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.015304394
Predicted Genre: Cubism
Actual Genres: ['Romanticism']
Correct Prediction: No
------------------------------
Test Sample 3:
Test Data Painter: Pierre-Auguste Renoir
Predicted Highest Probability: 0.005622839
Predicted Genre: Romanticism
Actual Genres: ['Impressionism']
Correct Prediction: No
------------------------------
Test Sample 4:
Test Data Painter: Gustave Courbet
Predicted Highest Probability: 0.000208387
Predicted Genre: Realism
Actual Genres: ['Realism']
Correct Prediction: Yes
------------------------------
Test Sample 5:
Test Data Painter: Francisco Goya
Predicted Highest Probability: 0.999998063
Predicted Genre: Romanticism
Actual Genres: ['Romanticism']
Correct

In [583]:
# Calculate and display accuracy
accuracy = correct_predictions / total_predictions * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 65.05%
